# Recession effect on Housing Prices from University Towns

This project is about examining the Hypothesis, **"The housing price from homes across university towns were less effected by the Great Recession".**

Glossary terms/Definitions used in this assignment:
* A _quarter_ is a specific three month period.
* A _recession_ is defined as the period which starts with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

The following data files are used for this project:
* ```City_Zhvi_AllHomes.csv```, obtained from [Zillow research data site](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), contains US median house sale prices at city level.
* A list of university towns in the United States can be obtained by webscraping the [Wikipedia page](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) on college towns which has been copied and pasted into the file ```university_towns.txt```.
* GDP over time of the United States in current US dollars, in quarterly intervals, can be obtained from [Bureau of Economic Analysis, US Department of Commerce](http://www.bea.gov/national/index.htm#gdp) which has been copied and pasted into the file ```ugdplev.xls```.

In [1]:
import pandas as pd # import pandas which is high-performance, easy-to-use data structures and data analysis framework
import numpy as np # import numpy which is fundamental package for scientific computing with Python
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = "../data_ds/"

In [3]:
# dictionary to map state names to two letter acronyms
state_acronyms = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
                  'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 
                  'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia',
                  'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 
                  'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam',
                  'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
                  'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 
                  'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 
                  'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 
                  'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
                  'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_university_towns_list():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list.'''
  
    data = pd.read_table(data_dir + 'university_towns.txt', header=None) # read university_towns.txt which contains a list of university towns in the United States and store it in a dataframe
    data['Cleaned Data'] = data[0].apply(lambda x: x.split(
    '(')[0].strip() if x.count('(') > 0 else x) # removing every character from "(" to the end. 
    data['RegionName'] = data['Cleaned Data'].apply(lambda x: x if x.count('[edit]') == 0 else np.NaN) #  replacing every state (ending with "[edit]") with NaN in a new column, "Region name",for easier selection of counties 
    data["State"] = 0 # initialise state column with 0
    state = None
    for index, entry in enumerate(data['Cleaned Data']): #assigning state names to regions
        if entry.count('[edit]') > 0:
           state = entry.split('[')[0].strip()
           data["State"][index] = state
        else:
           data["State"][index] = state
    data = data.drop(0, axis=1) #droping unwanted columns
    data = data.dropna() # dropping rows with null values
    data.index = list(range(len(data))) # reindexing from 0 to length of the datarfame
    columns_to_keep = ['State',
                       'RegionName'] 
    data = data[columns_to_keep] # keeping only columns of interest
    return data

In [5]:
get_university_towns_list().head(10)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [6]:
def get_recession_start_quarter():
    '''Returns the year and quarter of the recession start time.'''
    
    GDP_over_time = pd.ExcelFile(data_dir + 'gdplev.xls') # read university_towns.txt which contains quarterly GDP over time in the United States and store it in a dataframe with header excluded
    GDP_over_time = GDP_over_time.parse(GDP_over_time.sheet_names[0]) # parse the first sheet of the dataframe
    del GDP_over_time[GDP_over_time.columns[3]], GDP_over_time[GDP_over_time.columns[-1]] # delete unwanted columns
    GDP_over_time.drop(GDP_over_time.index[:7],inplace=True) # drop unwanted rows
    GDP_over_time = GDP_over_time.rename(index=str, columns={'Current-Dollar and "Real" Gross Domestic Product':'Year','Unnamed: 1':'GDP in billions of current dollars 1','Unnamed: 2':'GDP in billions of chained 2009 dollars 1','Unnamed: 4':'Quarter','Unnamed: 5':'GDP in billions of current dollars 2','Unnamed: 6':'GDP in billions of chained 2009 dollars 2'}) # renaming column headers
    millennium_first_quarter = GDP_over_time[GDP_over_time['Quarter']=='2000q1'].index[0] # extracting index of millennium start quarter
    GDP_over_time = GDP_over_time.loc[millennium_first_quarter:] # extracting the dataframe from millennium start quarter
    del GDP_over_time['Year'], GDP_over_time['GDP in billions of current dollars 1'], GDP_over_time['GDP in billions of chained 2009 dollars 1'] # deleting unwanted columns
    GDP_over_time.index = list(range(len(GDP_over_time))) # reindexing from 0 to length of the datarfame
    Quarterly_GDP = 0.0
    GDP_Decline_Counter = 0
    recession_start_time = None
    for index, entry in enumerate(GDP_over_time['GDP in billions of current dollars 2']): # extracting recession start quarter
        if entry > Quarterly_GDP:
            Quarterly_GDP = entry
        elif GDP_Decline_Counter==2:
            Quarterly_GDP = entry
            recession_start_time = GDP_over_time['Quarter'][int(index)-3]
        else:
            Quarterly_GDP = entry
            GDP_Decline_Counter += 1
    return recession_start_time # returning recession start quarter

In [7]:
get_recession_start_quarter()

'2008q3'

In [8]:
def get_recession_end_quarter():
    '''Returns the year and quarter of the recession end time.'''
    
    GDP_over_time = pd.ExcelFile(data_dir + 'gdplev.xls') # read university_towns.txt which contains quarterly GDP over time in the United States and store it in a dataframe with header excluded
    GDP_over_time = GDP_over_time.parse(GDP_over_time.sheet_names[0]) # parse the first sheet of the dataframe
    del GDP_over_time[GDP_over_time.columns[3]], GDP_over_time[GDP_over_time.columns[-1]] # delete unwanted columns
    GDP_over_time.drop(GDP_over_time.index[:7],inplace=True) # drop unwanted rows
    GDP_over_time = GDP_over_time.rename(index=str, columns={'Current-Dollar and "Real" Gross Domestic Product':'Year','Unnamed: 1':'GDP in billions of current dollars 1','Unnamed: 2':'GDP in billions of chained 2009 dollars 1','Unnamed: 4':'Quarter','Unnamed: 5':'GDP in billions of current dollars 2','Unnamed: 6':'GDP in billions of chained 2009 dollars 2'}) # renaming column headers
    millennium_first_quarter = GDP_over_time[GDP_over_time['Quarter']=='2000q1'].index[0] # extracting index of millennium start quarter
    GDP_over_time = GDP_over_time.loc[millennium_first_quarter:] # extracting the dataframe from millennium start quarter
    del GDP_over_time['Year'], GDP_over_time['GDP in billions of current dollars 1'], GDP_over_time['GDP in billions of chained 2009 dollars 1'] # deleting unwanted columns
    GDP_over_time.index = list(range(len(GDP_over_time))) # reindexing from 0 to length of the datarfame
    Quarterly_GDP = 0.0
    GDP_Decline_Counter = 0
    recession_start_time = None
    for index, entry in enumerate(GDP_over_time['GDP in billions of current dollars 2']): # extracting recession end quarter
        if entry > Quarterly_GDP:
            Quarterly_GDP = entry
        elif GDP_Decline_Counter==2:
            Quarterly_GDP = entry
            recession_start_time = GDP_over_time['Quarter'][int(index)-3]
        else:
            Quarterly_GDP = entry
            GDP_Decline_Counter += 1
    
    recession_start_quarter = GDP_over_time[GDP_over_time['Quarter']==recession_start_time].index[0]
    GDP_over_time_revised = GDP_over_time.loc[recession_start_quarter:]
    GDP_over_time_revised.index = list(range(len(GDP_over_time_revised)))
    
    Quarterly_GDP = GDP_over_time_revised['GDP in billions of current dollars 2'][0]
    GDP_Growth_Counter = 0
    recession_end_time = 0
    for index, entry in enumerate(GDP_over_time_revised['GDP in billions of current dollars 2']):
        if GDP_Growth_Counter == 2:
            Quarterly_GDP = entry
            recession_end_time = GDP_over_time_revised['Quarter'][int(index-1)]
            break
        elif entry > Quarterly_GDP:
            Quarterly_GDP = entry
            GDP_Growth_Counter += 1
        else:
            Quarterly_GDP = entry
        
    return recession_end_time # returning recession end quarter

In [9]:
get_recession_end_quarter()

'2009q4'

In [10]:
def get_recession_bottom_quarter():
    '''Returns the year and quarter of the recession bottom time.'''
    
    GDP_over_time = pd.ExcelFile(data_dir + 'gdplev.xls') # read university_towns.txt which contains quarterly GDP over time in the United States and store it in a dataframe with header excluded
    GDP_over_time = GDP_over_time.parse(GDP_over_time.sheet_names[0]) # parse the first sheet of the dataframe
    del GDP_over_time[GDP_over_time.columns[3]], GDP_over_time[GDP_over_time.columns[-1]] # delete unwanted columns
    GDP_over_time.drop(GDP_over_time.index[:7],inplace=True) # drop unwanted rows
    GDP_over_time = GDP_over_time.rename(index=str, columns={'Current-Dollar and "Real" Gross Domestic Product':'Year','Unnamed: 1':'GDP in billions of current dollars 1','Unnamed: 2':'GDP in billions of chained 2009 dollars 1','Unnamed: 4':'Quarter','Unnamed: 5':'GDP in billions of current dollars 2','Unnamed: 6':'GDP in billions of chained 2009 dollars 2'}) # renaming column headers
    millennium_first_quarter = GDP_over_time[GDP_over_time['Quarter']=='2000q1'].index[0] # extracting index of millennium start quarter
    GDP_over_time = GDP_over_time.loc[millennium_first_quarter:] # extracting the dataframe from millennium start quarter
    del GDP_over_time['Year'], GDP_over_time['GDP in billions of current dollars 1'], GDP_over_time['GDP in billions of chained 2009 dollars 1'] # deleting unwanted columns
    GDP_over_time.index = list(range(len(GDP_over_time))) # reindexing from 0 to length of the datarfame
    Quarterly_GDP = 0.0
    GDP_Decline_Counter = 0
    recession_start_time = None
    for index, entry in enumerate(GDP_over_time['GDP in billions of current dollars 2']): # extracting recession bottom quarter
        if entry > Quarterly_GDP:
            Quarterly_GDP = entry
        elif GDP_Decline_Counter==2:
            Quarterly_GDP = entry
            recession_start_time = GDP_over_time['Quarter'][int(index)-3]
        else:
            Quarterly_GDP = entry
            GDP_Decline_Counter += 1
    
    recession_start_quarter = GDP_over_time[GDP_over_time['Quarter']==recession_start_time].index[0]
    GDP_over_time_revised = GDP_over_time.loc[recession_start_quarter:]
    GDP_over_time_revised.index = list(range(len(GDP_over_time_revised)))
    
    Quarterly_GDP = GDP_over_time_revised['GDP in billions of current dollars 2'][0]
    GDP_Growth_Counter = 0
    recession_end_time = 0
    for index, entry in enumerate(GDP_over_time_revised['GDP in billions of current dollars 2']): # extracting recession bottom quarter
        if GDP_Growth_Counter == 2:
            Quarterly_GDP = entry
            recession_end_time = GDP_over_time_revised['Quarter'][int(index-1)]
            break
        elif entry > Quarterly_GDP:
            Quarterly_GDP = entry
            GDP_Growth_Counter += 1
        else:
            Quarterly_GDP = entry

    recession_end_quarter = GDP_over_time_revised[GDP_over_time_revised['Quarter']==recession_end_time].index[0]
    GDP_over_time_revised = GDP_over_time_revised.loc[:recession_end_quarter]
    argument_minimum = np.argmin(GDP_over_time_revised['GDP in billions of current dollars 2'], axis=0)[0]
    return GDP_over_time_revised['Quarter'][argument_minimum] # returning recession bottom quarter

In [11]:
get_recession_bottom_quarter()

'2009q2'

In [12]:
def convert_housing_data_to_mean_quarters_price():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe.'''
    housing_data = pd.read_csv(data_dir + 'City_Zhvi_AllHomes.csv') # read City_Zhvi_AllHomes.csv which contains median home sale prices and store it in a dataframe
    housing_data.drop(housing_data.columns[6:51],axis=1, inplace=True) # frod unwanted year columns
    
    for index_2 in range(int(67)): # data preprocessing for renaming quarter columns
        index_quarter = str(2000 + divmod(index_2, 4)[0])+"q"+str(((index_2)%4)+1)
        housing_data[index_quarter] = 0
    
    housing_data_revised = housing_data.copy() # making a copy of the dataframe for help in renaming quarter columns
    housing_data.drop(housing_data.columns[-67:],axis=1, inplace=True)
    housing_data_revised.drop(housing_data_revised.columns[6:-67],axis=1,inplace=True)
    
    for index_2 in range(int(len(housing_data_revised.columns[6:]))): # data preprocessing for renaming quarter columns
        housing_data_revised[housing_data_revised.columns[6:][index_2]] = housing_data[housing_data.columns[6+(index_2*3):6+(index_2*3)+3]].mean(axis=1)
    
    housing_data_revised = housing_data_revised.sort_values('State', ascending=True) # sort by state name in alphabetical order 
    housing_data_revised = housing_data_revised.replace(state_acronyms) # replace state values from two letter abbreviation to full state name according to aforementioned dictionary
    housing_data_revised = housing_data_revised.set_index(['State','RegionName']) # setting a multi index of state and county
    housing_data_revised.drop(housing_data_revised.columns[:4],axis=1,inplace=True) # droping unwanted columns
    return housing_data_revised

In [13]:
convert_housing_data_to_mean_quarters_price().head(10)

2000q1         2000q2         2000q3  \
State  RegionName                                                  
Alaska Kenai                   NaN            NaN            NaN   
       Ketchikan               NaN            NaN            NaN   
       Anchor Point            NaN            NaN            NaN   
       Anchorage     174633.333333  175266.666667  179566.666667   
       Palmer        168766.666667  170566.666667  174233.333333   
       Seward                  NaN            NaN            NaN   
       Kodiak                  NaN            NaN            NaN   
       Tanaina       160500.000000  163500.000000  166366.666667   
       Lakes         172733.333333  174133.333333  177266.666667   
       North Pole    157466.666667  159100.000000  162133.333333   

                            2000q4         2001q1         2001q2  \
State  RegionName                                                  
Alaska Kenai                   NaN            NaN            NaN   
       Ketchikan               NaN            NaN            NaN   
       Anchor Point            NaN            NaN            NaN   
       Anchorage     182833.333333  182766.666667  183933.333333   
       Palmer        175533.333333  169466.666667  163233.333333   
       Seward                  NaN            NaN            NaN   
       Kodiak                  NaN            NaN            NaN   
       Tanaina       168033.333333  158766.666667  149200.000000   
       Lakes         179133.333333  172400.000000  166000.000000   
       North Pole    165500.000000  159433.333333  149766.666667   

                            2001q3         2001q4         2002q1  \
State  RegionName                                                  
Alaska Kenai                   NaN            NaN            NaN   
       Ketchikan               NaN            NaN            NaN   
       Anchor Point            NaN            NaN            NaN   
       Anchorage     188566.666667  191866.666667  193966.666667   
       Palmer        166933.333333  168933.333333  165300.000000   
       Seward                  NaN            NaN            NaN   
       Kodiak                  NaN            NaN            NaN   
       Tanaina       150966.666667  151600.000000  153033.333333   
       Lakes         169500.000000  170766.666667  168766.666667   
       North Pole    151266.666667  151000.000000  150033.333333   

                            2002q2  ...         2014q2         2014q3  \
State  RegionName                   ...                                 
Alaska Kenai                   NaN  ...  199466.666667  197966.666667   
       Ketchikan               NaN  ...  196866.666667  197100.000000   
       Anchor Point            NaN  ...  157800.000000  158000.000000   
       Anchorage     196700.000000  ...  278200.000000  280766.666667   
       Palmer        164800.000000  ...  225566.666667  228766.666667   
       Seward                  NaN  ...  194600.000000  190533.333333   
       Kodiak                  NaN  ...  256400.000000  256966.666667   
       Tanaina       156100.000000  ...  208900.000000  211000.000000   
       Lakes         169900.000000  ...  228200.000000  233966.666667   
       North Pole    151866.666667  ...  199933.333333  199400.000000   

                            2014q4         2015q1         2015q2  \
State  RegionName                                                  
Alaska Kenai         194600.000000  198500.000000  202700.000000   
       Ketchikan     197700.000000  198033.333333  196300.000000   
       Anchor Point  155566.666667  154266.666667  154100.000000   
       Anchorage     281700.000000  284166.666667  287166.666667   
       Palmer        229366.666667  234466.666667  237800.000000   
       Seward        186600.000000  186800.000000  185533.333333   
       Kodiak        257933.333333  258566.666667  256133.333333   
       Tanaina       212466.666667  215200.000000  215633.333333   
       Lakes        

In [14]:
def ttest():
    '''First the new data showing the decline or growth of housing prices between the recession start 
    and recession botton. Then a ttest is ran to compare the university town values to the non-university
    town values, return whether the alternative hypotheses is true or not. The p-value of confidence is 
    then computed.
    
    A tuple (different, p, better) is returned where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing_data_revised = convert_housing_data_to_mean_quarters_price() # obtain mean quarterly housing values from the above function
    housing_data_revised = housing_data_revised[housing_data_revised.columns[33:-29]] # extract mean quarterly housing values from only the recession period
    
    university_towns = get_university_towns_list() #obtain list of university towns form the above function
    university_towns = university_towns.sort_values('State', ascending=True) # sort university towns by state name in alphabetical order
    university_towns = university_towns.set_index(['State','RegionName']) # set state name and county as multi-index
    
    university_town_values = pd.merge(university_towns, housing_data_revised, how='inner', left_index=True, right_index=True) # obtain mean quarterly housing values in university towns
    non_university_town_values = housing_data_revised[~housing_data_revised.index.isin(university_town_values.index)] # obtain mean quarterly values of housing in non-university towns

    university_town_mean_ratio = ((university_town_values['2008q2'])/(university_town_values['2009q2'])).mean() # obtain ratio of mean quarterly housing values in recession start to recession bottom in university towns
    non_university_town_mean_ratio = ((non_university_town_values['2008q2'])/(non_university_town_values['2009q2'])).mean() # obtain ratio of mean quarterly housing values in recession start to recession bottom in non-university towns
    
    from scipy import stats

    p = stats.ttest_ind((university_town_values['2008q2'])/(university_town_values['2009q2']), (non_university_town_values['2008q2'])/(non_university_town_values['2009q2']), nan_policy='omit')[1] # run t-test on the above ratios of university and non-university towns
    different = p<0.01
    better = "university town" if (university_town_mean_ratio < non_university_town_mean_ratio) else "non-university town"
    
    return (different, p, better)

In [15]:
ttest()

(True, 0.002724063704753125, 'university town')